In [3]:
from lusidtools.jupyter_tools import toggle_code

"""Compliance in LUSID

Attributes
----------
Compliance
OMS
Pre/Post trade checks
"""

toggle_code("Toggle Docstring")

## Configuring and Running Pre-Trade Compliance
In this example we demonstrate the configuration of several simple pre-trade rules and run them against a sample portfolio. Then we will raise some orders on that portfolio and re-check pre-trade compliance. Lastly, we will allocate partially against the newly created order and run a post-trade compliance. 

## Imports

In [25]:
import lusid
import lusid.api as la
import lusid.models as lm
from lusid.models.upsert_compliance_rule_request import UpsertComplianceRuleRequest
from lusid.models.reference_list_request import ReferenceListRequest
from lusid import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response,
)
from collections import defaultdict
import pandas as pd
import numpy as np
import json
import openpyxl
import os
import datetime
from datetime import datetime, timedelta, time, date
import pytz

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print("LUSID Environment Initialised")
print(
    "API Version: ",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version,
)

LUSID Environment Initialised
API Version:  0.6.12628.0


In [58]:
# define some APIs
properties_api = api_factory.build(la.PropertyDefinitionsApi)
referencelist_api = api_factory.build(la.ReferenceListsApi)
compliance_api = api_factory.build(la.ComplianceApi)
aggregation_api = api_factory.build(la.AggregationApi)
configuration_recipe_api = api_factory.build(la.ConfigurationRecipeApi)
portfolio_groups_api = api_factory.build(la.PortfolioGroupsApi)
portfolios_api = api_factory.build(la.PortfoliosApi)
instruments_api = api_factory.build(la.InstrumentsApi)
allocations_api = api_factory.build(lusid.api.AllocationsApi)
transaction_portfolios_api = api_factory.build(la.TransactionPortfoliosApi)
quotes_api = api_factory.build(la.QuotesApi)


In [27]:
# Specify a unique scope and code to segregate data in this tutorial from others
scope = "simplecompliance"
portfolio_code='EQUITY_UK'
portfolio_group_code='EQUITY_UK_GROUP'
portfolio_base_currency='USD'
lookthrough_portfolio='Lookthrough_fund'

## 1. Create instruments, portfolio and transactions to work with


### 1.0 Load transaction data

In [28]:
df = pd.read_csv("data/equity_transactions.csv")
df

,ISIN,sedol,instrument_type,instrument_id,name,sector,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,strategy,cash_transactions
0,GB0002162385,216238,equity,EQ_1234,Aviva,Financial Services,txn-1,StockIn,02/01/2020,04/01/2020,120000,4.23,600000,GBP,ftse_tracker,NaN
1,GB00BH0P3Z91,BH0P3Z9,equity,EQ_1235,BHP,Mining,txn-2,StockIn,02/01/2020,04/01/2020,60000,17.89,1080000,GBP,ftse_tracker,NaN
2,GB0031348658,3134865,equity,EQ_1236,Barclays,Financial Services,txn-3,StockIn,02/01/2020,04/01/2020,150000,1.80,300000,GBP,ftse_tracker,NaN
3,GB0007980591,798059,equity,EQ_1237,BP,Oil and Gas,txn-4,StockIn,02/01/2020,04/01/2020,100000,4.75,500000,GBP,ftse_tracker,NaN
4,GB0005405286,540528,equity,EQ_1238,HSBC,Financial Services,txn-5,StockIn,02/01/2020,04/01/2020,20000,5.89,120000,GBP,ftse_tracker,NaN
5,GB0006043169,604316,equity,EQ_1239,Morrisons,Food and Drink,txn-6,StockIn,15/01/2020,17/01/2020,180000,1.87,360000,GBP,ftse_tracker,NaN
6,GB0008847096,884709,equity,EQ_1240,Tesco,Food and Drink,txn-7,StockIn,16/01/2020,18/01/2020,4000,2.47,36000,GBP,ftse_tracker,NaN
7,GB00BGDT3G23,BGDT3G2,equity,EQ_1241,Rightmove,Real Estate,txn-8,StockIn,15/01/2020,17/01/2020,80000,6.59,480000,GBP,ftse_tracker,NaN
8,GB00BH4HKS39,BH4HKS3,equity,EQ_1242,vodafone,Telecommunications,txn-9,StockIn,15/01/2020,17/01/2020,450000,1.56,450000,GBP,ftse_tracker,NaN
9,GB00B1XZS820,B1XZS82,equity,EQ_1243,Anglo American plc,Mining,txn-10,StockIn,15/01/2020,17/01/2020,35000,21.68,700000,GBP,ftse_tracker,NaN


### 1.1 Load instruments
loaded from df file

In [29]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Isin": "ISIN",
        "Sedol": "sedol",
    },
    "required": {"name": "name"},
}

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
    property_columns=["sector"],
)
succ, failed, errors = format_instruments_response(result)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

,success,failed,errors
0,11,0,0


### 1.2 Create portfolio

In [40]:
def create_portfolio(scope, portfolio_code, name):

    pf_df = pd.DataFrame(
        data=[
            {"portfolio_code": portfolio_code, "portfolio_name": name},
        ]
    )

    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "$USD",
        },
        "optional": {"created": "$2020-01-01T00:00:00+00:00"},
    }

    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
    )

    succ, failed = format_portfolios_response(result)
    display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}]))

In [42]:
#create portfolio and lookthrough portfolio
create_portfolio(scope, portfolio_code, "LUSID's top 10 FTSE stock portfolio")
create_portfolio(scope, lookthrough_portfolio, "Lookthrough portfolio")


,success,failed
0,1,0


,success,failed
0,1,0


### 1.3 Create portfolio group

In [43]:
def create_portfolio_group(portfolio_groups_api, scope, code, portfolios):

    portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=5000)

    try:
        portfolio_groups_api.delete_portfolio_group(scope=scope, code=code)
    except:
        pass

    group_request = lm.CreatePortfolioGroupRequest(
        code=code,
        display_name=code,
        values=portfolios,
        sub_groups=None,
        description=None,
        created=portfolio_creation_date,
    )

    portfolio_group = portfolio_groups_api.create_portfolio_group(
        scope=scope, create_portfolio_group_request=group_request
    )

    return portfolio_group

In [121]:
result = create_portfolio_group(portfolio_groups_api, scope, portfolio_group_code, [lm.ResourceId(scope=scope, code=lookthrough_portfolio)])


### 1.4 Load transactions

In [45]:
transaction_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
    },
    "required": {
        "code": f"${portfolio_code}",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": f"${portfolio_base_currency}",
        "settlement_date": "txn_settle_date",
    },
    "optional": {},
    "properties": [],
}

result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
    properties_scope=scope,
)
succ, failed = format_transactions_response(result)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

,success,failed,errors
0,1,0,0


### 1.5 Load quotes

In [46]:
quotes_df = pd.read_csv("data/equity_quotes.csv")
# Compliance runs at latest asat
quotes_df["quote_date"] = quotes_df["quote_date"].apply(
    lambda s: datetime.now(pytz.UTC).strftime("%d-%b-%y")
)
quotes_df

,ISIN,quote_date,bid,mid,ask
0,GB0002162385,21-Feb-24,2.29,2.30,2.31
1,GB00BH0P3Z91,21-Feb-24,12.81,12.87,12.93
2,GB0031348658,21-Feb-24,0.88,1.88,0.88
3,GB0007980591,21-Feb-24,3.06,3.08,3.10
4,GB0005405286,21-Feb-24,4.00,4.02,4.04
5,GB0006043169,21-Feb-24,1.87,1.88,1.89
6,GB0008847096,21-Feb-24,2.35,2.36,2.37
7,GB00BGDT3G23,21-Feb-24,4.63,4.65,4.67
8,GB00BH4HKS39,21-Feb-24,1.08,1.09,1.10
9,GB00B1XZS820,21-Feb-24,13.90,13.97,14.04


In [47]:
quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$Isin",
    "quote_id.effective_at": "quote_date",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "ISIN",
    "metric_value.unit": "$USD",
}


quotes_mapping["quote_id.quote_series_id.field"] = "$mid"
quotes_mapping["metric_value.value"] = "mid"
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=quotes_df,
    mapping_required=quotes_mapping,
    mapping_optional={},
    file_type="quotes",
)
succ, failed, errors = format_quotes_response(result)
display(
    pd.DataFrame(
        data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
    )
)

,success,failed,errors
0,12,0,0


### 1.6 Securitise the instrument and add the fund to the lookthrough

In [87]:
upsert_instrument = instruments_api.upsert_instruments(
    request_body={
        "child instrument": lm.InstrumentDefinition(
            name="child instrument",
            identifiers={"ClientInternal": lm.InstrumentIdValue(value="ChildInstrument")},
            look_through_portfolio_id={'scope':scope,'code':portfolio_code},
            definition={'instrumentType':'SimpleInstrument',
                        'domCcy':'USD',
                        'assetClass':'Equities',
                        'simpleInstrumentType':'EquityIndex'}),
    }
)
upsert_instruments_response_df = lusid_response_to_data_frame(list(upsert_instrument.values.values()))
display(upsert_instruments_response_df[["name", "lusid_instrument_id"]])

,name,lusid_instrument_id
0,child instrument,LUID_00003DA3


In [88]:
# Set trade variables
trade_date = datetime(2024, 2, 16, tzinfo=pytz.utc)
settle_days = 2
units = 10

# Book a StockIn transaction against the forward
fwd_txn = lm.TransactionRequest(
    transaction_id="TXN001",
    type="StockIn",
    instrument_identifiers={"Instrument/default/ClientInternal": 'ChildInstrument'},
    transaction_date=trade_date.isoformat(),
    settlement_date=(trade_date + timedelta(days=settle_days)).isoformat(),
    units=units,
    transaction_price=lm.TransactionPrice(price=200,type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=10000,currency="GBP"),
    exchange_rate=1,
    transaction_currency="GBP"
)

response = transaction_portfolios_api.upsert_transactions(scope=scope,
                                                    code=lookthrough_portfolio,
                                                    transaction_request=[fwd_txn])

print(f"Transaction successfully updated at time: {response.version.as_at_date}")

Transaction successfully updated at time: 2024-02-21 14:46:23.333587+00:00


In [89]:
def upsert_quote(ClientInternal, date, rate):
    quote_request = {
       "quote_request" : lm.UpsertQuoteRequest(
            quote_id=lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=ClientInternal,
                    instrument_id_type="ClientInternal",
                    quote_type="Price",
                    field="mid",
                ),
                effective_at=date.isoformat(),
            ),
            metric_value=lm.MetricValue(value=rate, unit="USD"),
        )}

    # Upsert the quotes into LUSID
    response = quotes_api.upsert_quotes(scope=scope, request_body=quote_request)
    display( f"Successfully upserted quote for instrument: {response.values['quote_request'].quote_id.quote_series_id.instrument_id} with date:{date}")

In [90]:
upsert_quote("ChildInstrument", trade_date, 30)


'Successfully upserted quote for instrument: ChildInstrument with date:2024-02-16 00:00:00+00:00'

### 1.7 Create recipes

In [84]:
# Create a recipe to perform a valuation for the lookthrough
configuration_recipe = lm.ConfigurationRecipe(
    scope=scope,
    code="lookthrough_mid",
    market=lm.MarketContext(
        market_rules=[
            # define how to resolve the quotes
            lm.MarketDataKeyRule(
                key="Quote.Isin.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",

            ),
        ],
        options=lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="Isin",
            default_scope=scope,
            attempt_to_infer_missing_fx=True,
        ),
    ),
    pricing=lm.PricingContext(
        options = lm.PricingOptions(
            model_selection = lm.ModelSelection(
            library = 'Lusid',
            model = 'SimpleStatic'
            ),
            allow_partially_successful_evaluation = True),
    
        # toggle look through
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="Equity",
                model_options=lm.IndexModelOptions(portfolio_scaling="Sum", model_options_type="IndexModelOptions"),
            ),
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="SimpleInstrument",
                model_options=lm.IndexModelOptions(portfolio_scaling="Sum", model_options_type="IndexModelOptions"),

            )
        ],
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=lm.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

In [97]:
# Create look-through-enabled recipe
lookthrough_config_recipe = lm.ConfigurationRecipe(
    scope=scope,
    code="lookthrough",
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Quote.*.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval= "12M.0D",
            ),
            lm.MarketDataKeyRule(
                key="FX.CurrencyPair.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Rate",
                field="mid",
                quote_interval= "12M.0D",
            ),
        ],
        options=lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="Isin",
            default_scope=scope,
            attempt_to_infer_missing_fx=True,
        ),
    ),
    pricing=lm.PricingContext(
        options={"AllowPartiallySuccessfulEvaluation": False},
        # toggle look through
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="SimpleInstrument",
                model_options=lm.IndexModelOptions(portfolio_scaling="Sum", model_options_type="IndexModelOptions"),
            )
        ],
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=lm.UpsertRecipeRequest(
            configuration_recipe=lookthrough_config_recipe
        )
    )
)

### 1.8 Run a test valuation

In [98]:
def generate_valuation_request(valuation_effectiveAt, price_field):

    # Create the valuation request
    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(
            scope=scope, code="lookthrough"
        ),
        metrics=[
            lm.AggregateSpec("Instrument/default/Name", "Value"),
            lm.AggregateSpec("Valuation/PvInReportCcy", "Proportion"),
            lm.AggregateSpec("Valuation/PvInReportCcy", "Sum"),
            lm.AggregateSpec("Holding/default/Units", "Sum"),
            lm.AggregateSpec("Aggregation/Errors", "Value"),
        ],
        group_by=["Instrument/default/Name"],
        filters=[lm.PropertyFilter('Instrument/default/Name', 'NotEquals', 'USD'),
                lm.PropertyFilter('Instrument/default/Name', 'NotEquals', 'USD Cash')
        ],
        portfolio_entity_ids=[
            lm.PortfolioEntityId(scope=scope, code=lookthrough_portfolio)
        ],
        valuation_schedule=lm.ValuationSchedule(
            effective_at=valuation_effectiveAt.isoformat()
        ),
    )

    return valuation_request

In [99]:
# we will use the mid price
# we can see that instead of valuing the fund, its valuing the underlying of the fund.
aggregation = aggregation_api.get_valuation(
    valuation_request=generate_valuation_request(datetime.now(pytz.UTC), "mid")
)
pd.DataFrame(aggregation.data)

,Instrument/default/Name,Proportion(Valuation/PvInReportCcy),Sum(Valuation/PvInReportCcy),Sum(Holding/default/Units),Aggregation/Errors
0,Aviva,0.080716,24.214845,10.528194,[]
1,BHP,0.225830,67.748926,5.264097,[]
2,Barclays,0.082471,24.741255,13.160242,[]
3,BP,0.090075,27.022364,8.773495,[]
4,HSBC,0.023513,7.053890,1.754699,[]
5,Morrisons,0.098965,29.689506,15.792290,[]
6,Rightmove,0.108791,32.637400,7.018796,[]
7,vodafone,0.143447,43.033991,39.480726,[]
8,Anglo American plc,0.142993,42.898002,3.070723,[]
9,IAG group plc,0.000439,0.131602,0.087735,[]


## 2. Define reference list


### 2.1 Create reference lists

We need a couple of empty lists to allow pass-through behaviour. Note that empty list passed to a filter step in compliance rule setup means that we disable this filter step. please refer to 3.1 for the logic behind this. 
https://support.lusid.com/knowledgebase/article/KA-02248/en-us

A reference list in LUSID is a repository of data that can contain:

- A series of strings; for example if you want to store a list of asset classes.
- A series of portfolios or portfolio groups; for example if you want to store a list of portfolios to exclude from a compliance rule.
- A series of instruments; for example if you want to store a list of instruments to exclude from a particular compliance rule.
- A series of address keys; for example if you want to store a list of instrument properties that should exist to pass a compliance check.
- A series of decimals; for example if you wanted to store a list of upper and lower bounds that an order should remain within.

All reference lists in LUSID have a scope and a code forming a unique ID. Currently, you can pass one or more reference list IDs into various LUSID compliance rules to ensure the rule checks against the values in the list. For example, you can:

- Set up a compliance rule that checks orders do not comprise of instruments of a particular set of asset classes.
- Set up a compliance rule that checks orders contain a list of required properties, such as the country of origin.
- Specify a reference list of portfolios within a compliance rule to exclude those in the list from the rule.


In [135]:

# we will add mining here and will exclude mining from the portfolio holding later on

request = ReferenceListRequest(
    id=lm.ResourceId(scope=scope, code="restricted-sectors"),
    name="restricted sectors list",
    description="No mining firms",
    tags=[],
    reference_list=lm.StringList(reference_list_type="StringList", values=["Mining"]),
)

empty_portfolioid_list_response = (
    referencelist_api.upsert_reference_list_with_http_info(
        reference_list_request=request
    )
)

## 3. Setup compliance rules

### 3.1 A simple PV contribution threshold check
Here we are creating a 'PercentCheck' template rule that is using standard variation. 

In standard variation, you will see 2 filtered properties that are mandatatory to populate. As mentioned previously, if the property is not populated with any value, such as placeholder property and the reference list is empty, then we effectively disable the filter setup.

In our example, for instance, the rule is effectively running as simple variation as in this format, you do not specify property filtering and portfolio exclusion

Note that the compliance rulkes can be configured pre-trade or post-trade. with pre-trade screening incoming orders for the given scope and protgroup. please refer to this page for a general overview of Lusid OMS. https://support.lusid.com/knowledgebase/article/KA-02107/en-us


In [139]:
pvInReportCcy = lm.AddressKeyComplianceParameter("passed_validation", compliance_parameter_type='AddressKeyComplianceParameter')
pvInReportCcy._value = "Valuation/PvInReportCcy"

upsert_compliance_rule_request = UpsertComplianceRuleRequest(
    id=lm.ResourceId(scope=scope, code='rule-1'),
    name='threshold-pv-check',
    description='Simple threshold PV check',
    template_id=lm.ResourceId(scope='system', code='PercentCheck'),
    variation='simple',
    portfolio_group_id=lm.ResourceId(scope=scope, code=portfolio_group_code),
    active=True,
    parameters={
        #Since our portgroup only contains one portfolio, our reference list did not exclude any portfolio. 
        "Metric": pvInReportCcy,
        #setting up bounds for the rules. here we set the upper bound as 10% and receive warning when a single holding reaches 3-% of the portfolio PV. 
        "UpperBound": lm.DecimalComplianceParameter(value="10",compliance_parameter_type='DecimalComplianceParameter'),
        "LowerBound": lm.DecimalComplianceParameter(value="-1",compliance_parameter_type='DecimalComplianceParameter'),
        "UpperWarning": lm.DecimalComplianceParameter(value="30",compliance_parameter_type='DecimalComplianceParameter'),
        "LowerWarning": lm.DecimalComplianceParameter(value="-1",compliance_parameter_type='DecimalComplianceParameter'),
        "GroupingPropertyKey": lm.PropertyKeyComplianceParameter(value=f"Instrument/default/Name",compliance_parameter_type='PropertyKeyComplianceParameter'),
    },
    properties={}
)

compliance_api.upsert_compliance_rule(upsert_compliance_rule_request=upsert_compliance_rule_request)

### 3.2 A restricted sector check

We created 'restricted-sectors' with Mining as the value. In this rule, we are creating a 'PropertyValueNotInList' template where we look at the sector of the instruments and if the instrument is in 'Mining', then this rule will flag. 

- standard-not-in-list: No instruments can have the value of 'PropertyKey' in 'ExclusivePropertyList'
- standard-in-list: Instruments must have the value of 'PropertyKey' in 'ExclusivePropertyList'

In [140]:
upsert_compliance_rule_request = UpsertComplianceRuleRequest(
    id=lm.ResourceId(scope=scope, code="rule-2"),
    name="restricted-sector-check",
    description="Simple sector restriction",
    template_id=lm.ResourceId(scope="system", code="PropertyValueNotInList"),
    variation="standard-not-in-list",
    portfolio_group_id=lm.ResourceId(scope=scope, code=portfolio_group_code),
    active=True,
    parameters={
        "PropertyKey": lm.PropertyKeyComplianceParameter(
            value=f"Instrument/simplecompliance2/sector",
            compliance_parameter_type="PropertyKeyComplianceParameter",
        ),
        "ExclusivePropertyList": lm.StringListComplianceParameter(
            value=lm.ResourceId(scope=scope, code="restricted-sectors"),
            compliance_parameter_type="StringListComplianceParameter",
        ),
    },
    properties={},
)

compliance_api.upsert_compliance_rule(
    upsert_compliance_rule_request=upsert_compliance_rule_request
)

## 4. Run Compliance Rules

Currently the API enables us to run compliance as at now, rather than for a specific date.

In [143]:
run_response = compliance_api.run_compliance(run_scope=scope,rule_scope=scope,is_pre_trade=True,recipe_id_scope='simplecompliance',recipe_id_code='lookthrough')
run_code = run_response.run_id.code

print(f"Compliance run {run_response.run_id.scope}/{run_response.run_id.code} completed.")
print(f"Instigated: {run_response.instigated_at}, completed: {run_response.completed_at}")

Compliance run simplecompliance/13fdee6c-58bd-425e-b791-e973079b8914 completed.
Instigated: 2024-02-21 16:03:13.893977+00:00, completed: 2024-02-21 16:03:16.892099+00:00


## 5. Analyse the compliance result¶
### 5.1 Identify rule-level failures
At a very coarse level we can determine which rules have been breached in this compliance run. We can also see a list of orders and portfolios affected by the breach.

The list of orders might well be wider than initially expected. This is because LUSID compliance makes no assumptions about which orders take precedence; it takes a holistic, contingent view about compliance taking into account all orders and existing positions.

Here you can see that rule-1 has failed, and one portfolio is affected by the failure. We'll drilldown into this rule result in the following section.

In [144]:
def rule_level_dataframe(run_summary):
    # Use the first result as a way of generating overall headers
    h = ["", "", "", "", ""]
    c = ["Rule", "Rule Description", "Status", "Affected Orders", "Affected Portfolios"]

    df = pd.DataFrame([c], columns=h)

    new_labels = pd.MultiIndex.from_arrays([df.columns, df.iloc[0]], names=["", ""])
    df = df.set_axis(new_labels, axis=1).iloc[1:]

    # Now build a row per result
    for d in run_summary.details:
        r = [
            f"{d.rule_id.scope}/{d.rule_id.code}",
            d.rule_description,
            d.status,
            len(d.affected_orders),
            len(d.affected_portfolios_details),
        ]

        df.loc[len(df)] = r

    return df

In [145]:
#the rule is running on underlying fund holdings rather than the lookthrough fund's holding. 
run_summary = compliance_api.get_decorated_compliance_run_summary(
    scope=scope, code=run_code
)

df = rule_level_dataframe(run_summary)

print(
    f"Rule-level results for run {run_summary.run_id.scope}/{run_summary.run_id.code}."
)
display(df)

Rule-level results for run simplecompliance/13fdee6c-58bd-425e-b791-e973079b8914.


\
                      Rule           Rule Description  Status Affected Orders   
0  simplecompliance/rule-1  Simple threshold PV check  Failed               4   
1  simpleCompliance/rule-1  Simple threshold PV check  Failed               0   
2  simplecompliance/rule-2  Simple sector restriction  Failed               0   
3  simpleCompliance/rule-2  Simple sector restriction  Failed               0   

                       
  Affected Portfolios  
0                   1  
1                   1  
2                   1  
3                   1

### 5.2 Drilldown into specific rule results

#### 5.2.1 Simple PV threshold breach

`rule-1` failed the compliance check overall. This rule is a simple 10% PV contribution threshold check often used to control portfolio diversification, so a failure indicates that one or more instruments contributed more than 10%. Taking a look at the drilldown data for this rule we can see that `BHP`, `Rightmove`, `vodafone` and `Anglo American plc` in the `EQUITY_UK` portfolio each contribute more than 10%.

The rule is structured to
- start with all holdings in portfolio group 
- filter out contributions *for portfolios in an excluded portfolios list*
- group contributions *by portfolio id*
- two filter steps, here configured not to exclude any contributions
- further group contributions for each portfolio *by Isin*
- finally, compare a pair of Results Used (Valuation/PVInReportCcy compared to portfolio-level Valuation/PVInReportCcy) *(checking that one is less than 10% of the other)*

This information is represented by the Lineage for each drilldown row; the Lineage can be used to get a high-level view of what's caused a rule breach, and a pointer to where to start more detailed investigations if needed.

In [150]:
rule_result1 = compliance_api.get_compliance_rule_result(run_scope=scope, run_code=run_code, rule_scope=scope, rule_code='rule-1')


filtered_breakdowns1 = [breakdown for breakdown in rule_result1.rule_result.rule_breakdown if getattr(breakdown, 'group_status', '') == 'Failed']
rule_result1_df=lusid_response_to_data_frame(filtered_breakdowns1)
rule_result1_df


,group_status,results_used.Valuation/PvInReportCcy,results_used.Portfolios.Valuation/PvInReportCcy,properties_used,missing_data_information,lineage.0.index,lineage.0.label,lineage.0.sub_label,lineage.0.info_type,lineage.0.information,lineage.1.index,lineage.1.label,lineage.1.sub_label,lineage.1.info_type,lineage.1.information,lineage.2.index,lineage.2.label,lineage.2.sub_label,lineage.2.info_type,lineage.2.information,lineage.3.index,lineage.3.label,lineage.3.sub_label,lineage.3.info_type,lineage.3.information
0,Failed,67.748728,300.0,{},[],0,Initial,Initial,,,1,Portfolios,simplecompliance/Lookthrough_fund,PortfolioId,simplecompliance/Lookthrough_fund,2,FinalGrouping,BHP,PropertyKey,Instrument/default/Name,3,Compare,BHP,,
1,Failed,32.637305,300.0,{},[],0,Initial,Initial,,,1,Portfolios,simplecompliance/Lookthrough_fund,PortfolioId,simplecompliance/Lookthrough_fund,2,FinalGrouping,Rightmove,PropertyKey,Instrument/default/Name,3,Compare,Rightmove,,
2,Failed,43.033866,300.0,{},[],0,Initial,Initial,,,1,Portfolios,simplecompliance/Lookthrough_fund,PortfolioId,simplecompliance/Lookthrough_fund,2,FinalGrouping,vodafone,PropertyKey,Instrument/default/Name,3,Compare,vodafone,,
3,Failed,42.897877,300.0,{},[],0,Initial,Initial,,,1,Portfolios,simplecompliance/Lookthrough_fund,PortfolioId,simplecompliance/Lookthrough_fund,2,FinalGrouping,Anglo American plc,PropertyKey,Instrument/default/Name,3,Compare,Anglo American plc,,
4,Failed,-1314.704334,300.0,{},[],0,Initial,Initial,,,1,Portfolios,simplecompliance/Lookthrough_fund,PortfolioId,simplecompliance/Lookthrough_fund,2,FinalGrouping,IAG group plc,PropertyKey,Instrument/default/Name,3,Compare,IAG group plc,,
5,Failed,1314.836813,300.0,{},[],0,Initial,Initial,,,1,Portfolios,simplecompliance/Lookthrough_fund,PortfolioId,simplecompliance/Lookthrough_fund,2,FinalGrouping,USD,PropertyKey,Instrument/default/Name,3,Compare,USD,,


### 5.2.2 Restricted sector breach
Rule-2 failed the compliance check overall. This rule prevents trade in a restricted set of sectors (specifically just Mining in this case), so a failure indicates that one or more holdings across the whole portfolio group are of an instrument from a restricted sector.

In [151]:
rule_result2 = compliance_api.get_compliance_rule_result(
    run_scope=scope, run_code=run_code, rule_scope=scope, rule_code="rule-2"
)

filtered_breakdowns2 = [
    breakdown
    for breakdown in rule_result2.rule_result.rule_breakdown
    if getattr(breakdown, "group_status", "") == "Failed"
]

rule_result2_df = lusid_response_to_data_frame(filtered_breakdowns2)
display(rule_result2_df[["group_status", "lineage.1.sub_label"]])

,group_status,lineage.1.sub_label
0,Failed,Instrument/simplecompliance2/sector=Mining
